In [1]:
import os

name_list = os.listdir(r"data/image/train")
with open(r"data/dataset.txt", "w") as f:
    for name in name_list:
        if name.split(".")[0] == "cat":
            f.write(name+";0\n")
        elif name.split(".")[0] == "dog":
            f.write(name+";1\n")

In [2]:
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

def img_resize(img_path, size):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = cv2.resize(img, size)
    return img
    
def load_image(file_names, batch_size):
    n = len(file_names)
    i = 0
    while True:
        images = []
        labels = []
        for _ in range(batch_size):
            if i == 0:
                np.random.shuffle(file_names)
            file_name = file_names[i]
            img_path = os.path.join(r"data/image/train", file_name.split(";")[0])
            x = img_resize(img_path, (224,224))
            y = file_name.split(";")[1]
            images.append(x)
            labels.append(y)
            i = (i+1)%n
        labels = to_categorical(np.array(labels), num_classes=2)
        images = np.array(images)
        yield (images, labels)  # 这里的生成器只能输出一个张量，有()而不是[]
        

In [3]:
from model.hw_AlexNet import alexNet
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, \
    ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import numpy as np

if __name__ == "__main__":
    # 处理输入
    file_names =[]
    with open("data/dataset.txt") as f:
        for line in f:
            file_names.append(line[:-1])
    total_len = len(file_names)
    num_val = int(total_len*0.1)
    num_train = total_len - num_val
    
    np.random.seed(38283)
    np.random.shuffle(file_names)
    np.random.seed(None)
    
    model = alexNet()
    
    summary = model.summary()

    log_dir = "logs/"
    
    # checkpoint_period1 = ModelCheckpoint(
    #                             log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    #                             monitor='acc', 
    #                             save_weights_only=False, 
    #                             save_best_only=True, 
    #                             period=3
    #                         )
    model_checkPoint = ModelCheckpoint(log_dir + "ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5",
                                      monitor="acc",
                                      save_best_only=True,
                                      save_weights_only=False,
                                      save_freq="epoch")
    reduce_lr = ReduceLROnPlateau(monitor="acc",
                                  factor=0.5,
                                  patience=3,
                                  verbose=1)
    early_stop = EarlyStopping(monitor="val_loss",
                               min_delta=0,
                               patience=10,
                               verbose=1)
    
    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    
    batch_size=128
    model.fit_generator(generator=load_image(file_names[:num_train], batch_size),
                        steps_per_epoch=max(1,num_train//batch_size),
                        validation_data=load_image(file_names[num_train:], batch_size),
                        validation_steps=max(1,num_val//batch_size),
                        epochs=50,
                        initial_epoch=0,
                        callbacks=(model_checkPoint, reduce_lr, early_stop)
                        )
    model.save_weights(log_dir+"last1.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 48)        17472     
_________________________________________________________________
batch_normalization (BatchNo (None, 54, 54, 48)        192       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 48)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 128)       153728    
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 192)       2

In [ ]:
13:40